<h1>Jamaica Stock Exchange Analysis  </h1>
<p> Basic anslysis data sciences information </p>
<ul> 
<li> Calculate Simple Rate of Return </li>
<li> Stock Risk calculation
<li> Adjustment of split stocks 2015-2018</li>
<li> Adjust for dividends to stock analysis </li>    
<li> Bollinger Band to stock analysis </li>        
<li> Calculate Rate of Return for Portfolio </li>
<li> Calculate Risk Variance for Portfolio </li>
<li> Sharpe stocks Calculation</li>
<li> JSE-Index vs Price Regression on stocks </li>
<li> Markowitz Portfolio effeciency </li>
<li> CAPM - Capital Asset Pricing Model </li>
</ul>
<p><b> #Todo </b><i>Predictive Model</i></p>
<p><b> #Todo </b><i>MACDs</i></p>


<p>&copy; 2018 Romayne Whyte (Ijasoft) <p>


In [1]:
import pandas as pd
import numpy as np
import os
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import dataset as dt
import statsmodels.api as sm
from scipy import stats
from scipy import linalg
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
%matplotlib inline

<h2>Pulling Informtion from CSV Files</h2>

In [2]:
#Pull the valid stock information
folder = 'project'
stkframe, divframe = dt.generate_stock_dataframe(folder)
df_ind_data =  dt.pull_indices(folder)

           Symbol  Close  split  dividend
Date                                     
2017-03-10    AMG   4.87    5.0       0.0
           Symbol  Close  split  dividend
Date                                     
2018-08-28   BPOW    4.7   10.0       0.0
           Symbol  Close  split  dividend
Date                                     
2017-09-18    CAR  10.87   10.0       0.0
           Symbol  Close  split  dividend
Date                                     
2018-09-05    DTL   2.65   10.0       0.0
           Symbol  Close  split  dividend
Date                                     
2017-04-18   JAMT   3.67    2.0       0.0
            Symbol  Close  split  dividend
Date                                      
2017-06-22  JETCON   5.66    4.0       0.0
           Symbol  Close  split  dividend
Date                                     
2017-05-31    KEX   15.0    5.0       0.0
            Symbol  Close  split  dividend
Date                                      
2017-05-19  KPREIT    8.3    

In [4]:
stkframe.head()

,CCC,1834,AFS,AMG,BIL,BPOW,BRG,CAC,CAR,CBNY,...,SJ,SML,SOS,SRA,SSLVC,SVL,TTECH,VMIL,WISYNCO,XFUND
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,9.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.40,NaN,NaN,NaN,NaN
2008-01-03,10.00,4.19,NaN,NaN,NaN,NaN,NaN,NaN,7.101,NaN,...,NaN,NaN,NaN,NaN,NaN,2.75,NaN,NaN,NaN,NaN
2008-01-04,10.00,4.05,NaN,NaN,NaN,NaN,3.02,NaN,7.800,0.01,...,NaN,NaN,NaN,NaN,NaN,2.75,NaN,NaN,NaN,NaN
2008-01-07,10.00,4.00,NaN,NaN,NaN,NaN,3.05,NaN,7.799,NaN,...,NaN,NaN,NaN,NaN,NaN,2.51,NaN,NaN,NaN,NaN
2008-01-08,10.45,4.00,NaN,NaN,NaN,NaN,3.05,NaN,7.400,NaN,...,NaN,NaN,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN


In [5]:
divframe.head()

,CCC,1834,AFS,AMG,BIL,BPOW,BRG,CAC,CAR,CBNY,...,SJ,SML,SOS,SRA,SSLVC,SVL,TTECH,VMIL,WISYNCO,XFUND
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2008-01-03,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2008-01-04,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2008-01-07,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2008-01-08,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


<h2> Filter the data to a Time Range </h2>
<p> This Allows to specify the time range analyse stocks </p>

In [ ]:
#filter the data for the 01 January 2018 to 31 December, 2018

startdate = '2016-01-01'
enddate = '2018-12-31'

stkframe=stkframe[startdate:enddate]
divframe = divframe[startdate:enddate]
df_ind_data = df_ind_data[startdate:enddate]

In [ ]:
stkframe.info()

In [ ]:
stkframe.head()

In [ ]:
stkframe.tail()

In [ ]:
#filter the data for the 01 January 2018 to 03 December, 2018
#df_ind_data=df_ind_data['2018-01-01': '2018-11-31']

df_ind_data.info()


<h2>Time for Some Financial Analysis</h2>

<h3> Analysis for the indicies <h3>

In [ ]:
#Jamica Indices Return 
ind_returns = np.log(df_ind_data / df_ind_data.shift(1)) 
ind_returns.head()

In [ ]:
ind_returns.tail()

In [ ]:
indicesplot = ((df_ind_data / df_ind_data.iloc[0]) * 100)
indicesplot.sort_index(axis=0).head()
indicesplot.plot(figsize=(15, 6),linewidth=1);
#plt.rcParams.update({'font.size': 12})
#(df_ind_data / df_ind_data.iloc[0] * 100).plot(figsize=(15, 6));
plt.show()


In [ ]:
#indicies Annual Rate of Return
annual_ind_returns = ind_returns.apply(lambda x: (x.mean()*250) if  x.count() > 250 else (x.mean() * x.count())) 
#annual_ind_returns = ind_returns.mean() * ind_returns.count()
round(annual_ind_returns[:] * 100,5)

<h3> Analysis for Jamaia Stocks </h3>

In [ ]:
# This is simple rate of returns 
#rate_return = (stkframe/stkframe.shift(1))-1
# same as 
#rate_return = stkframe.pct_change()

# This is simple rate of returns with dividends involved
#rate_return = ((stkframe+divframe)/stkframe.shift(1))-1

# This is rate of returns with log
#rate_return_div = np.log((stkframe+divframe)/stkframe.shift(1))
rate_return_div = np.log(stkframe/stkframe.shift(1))
rate_return = np.log(stkframe/stkframe.shift(1))
rate_return = rate_return.replace([np.inf, -np.inf], np.nan)
rate_return.head()

In [ ]:
rate_return.tail()

In [ ]:
#Sample of rate of return
sym = 'SJ'

rate_return[sym].plot()
plt.figure(figsize=(30,25), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 10})
plt.show()

In [ ]:




df_closing = stkframe[sym]
df_30days = df_closing.rolling(window=20).mean()
df_30std = df_closing.rolling(window=20).std()
df_upperband = df_30days + (df_30std * 2)
df_lowerband = df_30days - (df_30std * 2)

#df_30days = df_closing.rolling( 30 ).mean()
#df_expand = df_closing.expanding().mean()
#df_ewm = df_closing.ewm(alpha=0.03).mean()

fig = plt.figure(figsize=(30,10))
plt.rcParams.update({'font.size': 22})
fig.autofmt_xdate()
ax = fig.add_subplot(1,1,1)
ax.plot(df_closing.index, df_closing, label=sym)
ax.plot(df_30days.index, df_30days, label='30 days rolling') #simple moving average
ax.plot(df_upperband.index, df_upperband, label='Upper Band') #simple moving average
ax.plot(df_lowerband.index, df_lowerband, label='Lower Band') #simple moving average


#ax.plot(df_30days.index, df_30days, label='30 days rolling') #simple moving average
#ax.plot(df_expand.index, df_30days, label='Expanding Average')
#ax.plot(df_ewm.index, df_ewm, label='EWMA($\\alpha=.03$)') # Exponentially weighted moving average

ax.grid()
ax.legend(loc=0)
ax.set_xlabel('Date')
plt.title(sym+' Adjusted Closes & Rolling Averages to 2018')
plt.plot()

In [ ]:
# set style, empty figure and axes
plt.style.use('fivethirtyeight')
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)

# Get index values for the X axis for facebook DataFrame
x_axis = df_closing.index.get_level_values(0)

# Plot shaded 21 Day Bollinger Band for Facebook
ax.fill_between(x_axis, df_upperband, df_lowerband, color='grey')

# Plot Adjust Closing Price and Moving Averages
ax.plot(x_axis, df_closing, color='blue', lw=2)
ax.plot(x_axis, df_30days, color='black',label='30 days rolling', lw=2)

# Set Title & Show the Image
ax.set_title('30 Day Bollinger Band For '+ sym)
ax.set_xlabel('Date (Year/Month)')
ax.set_ylabel('Price(JMD)')
ax.legend()
plt.show();

In [ ]:
rate_return[sym].dropna().head(),rate_return[sym].tail()

In [ ]:
stkframe[sym].dropna().head(),stkframe[sym].tail()

In [ ]:

#Calculate the Average Rate of Return

avg_returns = rate_return_div.apply(lambda x: (x.mean()*250) if  x.count() > 250 else (x.mean() * x.count()))

# Drop all records with no application
avg_returns = avg_returns.dropna()
# Drop all record with 0 as rate of return
#avg_returns = avg_returns.drop(avg_returns[avg_returns == 0].index)
round(avg_returns[avg_returns > 0].sort_values(),5)*100

In [ ]:
#find the stocks that have split
round(avg_returns[avg_returns <= 0].sort_values(),5)*100

<h3> Calculate Stock Return Risk (Standard Deviation) </h3>

In [ ]:
stk_std_risk = rate_return.apply(lambda x: (x.std()* 250 ** 0.5 ) if  x.count() > 250 else (x.std() * x.count() ** 0.5))
round(stk_std_risk.sort_values(),5) * 100

<h3> Sharpe Ratio </h3>

In [ ]:
bondRiskFreeRate = 0.05
sharpe = (avg_returns - bondRiskFreeRate) / stk_std_risk
sharpe[sharpe > 0].sort_values() 

In [ ]:
sharpe[sharpe <= 0].sort_values() 

<h3> Stock Correlation </h3>

In [ ]:
rate_with_index = pd.merge(rate_return,ind_returns[['JSE-Index', 'JSE-Junior']],left_index=True,right_index=True)

#Variance of two stocks
rate_with_index[[sym,'JSE-Index']].var() 

In [ ]:
#Annualize the Variance
rate_with_index[[sym,'JSE-Index']].var() * 250

In [ ]:
#Covariance of two stock
rate_with_index[[sym,'JSE-Index']].cov() 

In [ ]:
#Annualize covariance of two stock
rate_with_index[[sym,'JSE-Index']].cov() * 250

In [ ]:
#Correlation of returns of two stock
rate_with_index[[sym,'JSE-Index']].corr()

<h3> Capital Asset Pricing Model <span>(CAPM)</span>  </h3>

In [ ]:
#Calculate the Beta of all stocks
asym = []
abeta = []
for sy in rate_return.T.index:    
    adjrate = rate_with_index[[sy,'JSE-Index']].dropna()
    annual_count = 250 if adjrate[sy].count() > 250 else adjrate[sy].count()    
    cov = adjrate.cov() * annual_count
    var = adjrate['JSE-Index'].var() * annual_count
    beta = cov.iloc[0,1]/var
    
    #rate = rate_with_index[[sy,'JSE-Index']].dropna()
    #x = rate[sy].tolist()
    #y = rate['JSE-Index'].tolist()        
    
    #x = np.reshape(x, (len(x), 1))
    #y = np.reshape(y, (len(y), 1))    
    #x1 = sm.add_constant(x)    
    #model = sm.OLS(y, x1).fit()                
    #beta = model.params[1]
    #(beta,alpha,r_value,p_value,std_err) = stats.linregress(x,y) 
    
    
    
    asym.append(sy)
    abeta.append(beta)


beta_series = pd.Series(np.array(abeta),index=asym)
beta_series[beta_series > 0].sort_values(ascending=False)

In [ ]:
beta_series[beta_series < 0].sort_values(ascending=False)

In [ ]:
# Pull the Equity Risk Premium for Jamaica
# http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html

EquityRiskPremium = 0.126

In [ ]:
# This is the CAPM for stock
round((bondRiskFreeRate + beta_series * EquityRiskPremium) * 100,4)

<h2> Stock Regression </h2>

In [ ]:
df_merge = pd.merge(stkframe,df_ind_data,left_index=True,right_index=True)
jseindex = df_merge[['JSE-Index',sym]].dropna()['JSE-Index'].tolist()
stock_list = df_merge[['JSE-Index',sym]].dropna()[sym].tolist()

In [ ]:
jseindex = np.reshape(jseindex, (len(jseindex), 1))
stock_list = np.reshape(stock_list, (len(stock_list), 1))

In [ ]:
jseindex[1:5]

In [ ]:
jseindex1 = sm.add_constant(jseindex)
reg = sm.OLS(stock_list, jseindex1).fit()

In [ ]:
print(reg.summary())

In [ ]:
#Regression on Price vs Time
regressor = LinearRegression()
regressor.fit(jseindex, stock_list)
plt.rcParams.update({'font.size': 9})
plt.scatter(jseindex,stock_list, color='blue', label= 'Actual Price')
plt.plot(jseindex, regressor.predict(jseindex), color='red', linewidth=3, label = 'Predicted Price') #plotting the line made by linear regression
plt.title(sym +' Linear Regression | JSE-Index vs. Price')
plt.legend()
plt.xlabel('JSE-Index')
plt.ylabel('Prices')
plt.show()

<h3> Prediction Model Stock</h3>

In [ ]:
#Splitting the dataset into the Training set and Test set
xtrain, xtest, ytrain, ytest = train_test_split(jseindex, stock_list, test_size=0.33, random_state=42)

xtrain = np.reshape(xtrain, (len(xtrain), 1))
ytrain = np.reshape(ytrain, (len(ytrain), 1))

xtest = np.reshape(xtest, (len(xtest), 1))
ytest = np.reshape(ytest, (len(ytest), 1))

regressor.fit(xtrain, ytrain)
plt.rcParams.update({'font.size': 9})

#Train Set Graph
plt.scatter(xtrain, ytrain, color='yellow', label= 'Actual Price') #plotting the initial datapoints
plt.plot(xtrain, regressor.predict(xtrain), color='blue', linewidth=3, label = 'Predicted Price') #plotting the line made by linear regression
plt.title('Trained Linear Regression | Time vs. Price')
plt.legend()
plt.xlabel('Date Integer')
plt.show()

#Test Set Graph
plt.scatter(xtest, ytest, color='yellow', label= 'Actual Price') #plotting the initial datapoints
plt.plot(xtest, regressor.predict(xtest), color='blue', linewidth=3, label = 'Predicted Price') #plotting the line made by linear regression
plt.title('Test Linear Regression | Time vs. Price')
plt.legend()
plt.xlabel('Date Integer')
plt.show()

<h2> Calculating Portfolio </h2>

In [ ]:
# Find Stock that has the same about of trade days
value = stkframe.count(axis=0)
value = value.index[stkframe.count(axis=0) == stkframe['CAR'].count()]
value

In [ ]:
#Create Porfolio by randomly selecting 6 Stocks
selectStock = []
for i in range(0,6):
    rand_indx = np.random.randint(len(value))
    selectStock.append(value[rand_indx])

selectStock = ['CAR','GWEST','JMMBGL','JSE','SCIJMD','VMIL']

In [ ]:
rate_return[selectStock].describe()

In [ ]:
plt.rcParams.update({'font.size': 9})
portfolio_compare = pd.DataFrame
portfolio_compare = stkframe[selectStock]
portfolio_compare['JSE-Index'] = df_ind_data['JSE-Index']
(portfolio_compare / portfolio_compare.iloc[0] * 100).plot(figsize = (15, 6),linewidth=1);
plt.show()
#df_ind_data['JSE-Index']
#portfolio_compare[selectStock].tail()

<h3> Calculate Portfolio Rate of Return </h3>

In [ ]:
#Equally weighted Portfolio
weights = np.array([0.381762259,0.005572008,0.007400742,0.027681738,0.03950554,0.538077713])

In [ ]:
pfolio_rate_return = str(round(np.dot(avg_returns[selectStock], weights),5) * 100) + "%"
'Returns on Porfolio :' + pfolio_rate_return 

<h3> Calculate Portfolio Risk </h3>

In [ ]:
# Calculate the covariance of the total portfolio
portfolio_var = np.dot(weights.T, np.dot(rate_return[selectStock].cov() * 250, weights))
'Variance :' + str(round(portfolio_var,5)*100) + '%'

In [ ]:
#Portfolio Volatilty
portfolio_vol = np.dot(weights.T, np.dot(rate_return[selectStock].cov() * 250, weights)) ** 0.5
'Volitility :' + str(round(portfolio_vol, 3)*100) + '%' 

<h3> Markowitz Portfolio effeciency Frontier </h3>

In [ ]:
(rate_return[selectStock].mean() * 250)*100,rate_return[selectStock].cov() * 250,rate_return[selectStock].corr()

In [ ]:
frontier_pfolio_return = []
frontier_pfolio_vol = []
frontier_pfolio_weight = []
frontier_pfolio_sharpe = []
for x in range(0, 50000):
    ef_weights = np.random.random(len(selectStock))
    ef_weights /= np.sum(ef_weights)
    preturn = np.dot(avg_returns[selectStock]*100, ef_weights)
    pvol = np.sqrt(np.dot(ef_weights.T, np.dot(rate_return[selectStock].cov() * 250, ef_weights)))*100
    psharpe = (preturn - bondRiskFreeRate) / pvol
    #add values to list
    frontier_pfolio_return.append(preturn)
    frontier_pfolio_vol.append(pvol)
    frontier_pfolio_weight.append(ef_weights)
    frontier_pfolio_sharpe.append(psharpe)
    
frontier_pfolio_return = np.array(frontier_pfolio_return)
frontier_pfolio_vol = np.array(frontier_pfolio_vol)
frontier_pfolio_weight = np.array(frontier_pfolio_weight)
frontier_pfolio_sharpe = np.array(frontier_pfolio_sharpe)


frontier_pfolio_return[:5],frontier_pfolio_vol[:5]

In [ ]:
frontier_pfolio_frame = pd.DataFrame({'Return':frontier_pfolio_return,'Volitile':frontier_pfolio_vol,'Sharpe':frontier_pfolio_sharpe})

# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter,symbol in enumerate(selectStock):
    frontier_pfolio_frame[symbol+' weight'] = [weight[counter] for weight in frontier_pfolio_weight]
    
# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = frontier_pfolio_frame['Volitile'].min()
max_sharpe = frontier_pfolio_frame['Sharpe'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = frontier_pfolio_frame.loc[frontier_pfolio_frame['Sharpe'] == max_sharpe]
min_variance_port = frontier_pfolio_frame.loc[frontier_pfolio_frame['Volitile'] == min_volatility]

frontier_pfolio_frame=frontier_pfolio_frame.sort_values(by=['Volitile'],ascending=False)
frontier_pfolio_frame.head()

In [ ]:
frontier_pfolio_frame.tail()

In [ ]:
plt.style.use('seaborn-dark')
frontier_pfolio_frame.plot(x='Volitile',y='Return',c='Sharpe',kind='scatter',cmap='RdYlGn', edgecolors='black',
                           figsize=(10,8),grid=True);

plt.scatter(x=sharpe_portfolio['Volitile'], y=sharpe_portfolio['Return'], c='red', marker='D', s=200)
plt.scatter(x=min_variance_port['Volitile'], y=min_variance_port['Return'], c='blue', marker='D', s=200 )

plt.xlabel('Expected Volitility')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
# Red Dot portfolio with best Sharpe ratio (Best returns for risk)
sharpe_portfolio

In [ ]:
# Blue Dot stock with least varience / risk
min_variance_port